In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt 
import jupyros as jr
import rospy
from nav_msgs.msg import Odometry 
from std_msgs.msg import Bool
import ipywidgets as widgets
from ipywidgets import interact, interactive
from IPython.display import display
import assignment_2_2023.msg
from assignment_2_2023.msg import PlanningAction
from assignment_2_2023.msg import Info
import actionlib
import actionlib.msg
from matplotlib.widgets import TextBox


In [2]:
def on_feedback(action_feedback):
    global canCancel, send_coordinates
    
    # We can preempt the robot only if the state machine is in the "ACTIVE" status
    canCancel = (action_feedback.status.status == action_feedback.status.ACTIVE)
    if(canCancel):
        send_coordinates.description = "Cancel"
    else:
        send_coordinates.description = "Send Coordinates"

In [3]:
def on_result(action_result):
    global canCancel, send_coordinates
    # We can preempt the robot only if the state machine is not in the "SUCCEEDED" or in the "PREEMPTED" status
    canCancel = not(action_result.status.status == action_result.status.SUCCEEDED or action_result.status.status == action_result.status.PREEMPTED)
    if(canCancel):
        send_coordinates.description = "Cancel"
    else:
        send_coordinates.description = "Send Coordinates"
    

In [8]:
def on_odom(msg):
    global info, hbox_1, hbox_2, info_old
    pos = msg.pose.pose.position
	
	# Get the current velocity
    vel_linear_x = msg.twist.twist.linear.x
    vel_angular_z = msg.twist.twist.angular.z
    
    # rospy.loginfo("Velocity: %f, %f", vel_linear_x, vel_angular_z)
    
    # Prepare and publish the info message
    info = Info()
    info.x = pos.x
    info.y = pos.y
    info.vel_linear_x = vel_linear_x
    info.vel_angular_z = vel_angular_z
    if(abs(info_old.x-info.x) > 0.1 or abs(info_old.y-info.y) > 0.1):
        hbox_1.children = ([widgets.Label("{:.1f}".format(info.x)),])
        hbox_2.children = ([widgets.Label("{:.1f}".format(info.y)),])
        info_old = info

In [5]:

def on_coordinates_sent(b):
    global x_coordinate, y_coordinate, canCancel, pub_coordinates, pub_cancel, client, info, vbox
    if(b.description == "Cancel"):
        b.description = "Send Coordinates"
        client.cancel_goal()
        
        
        hbox = widgets.HBox()

        hbox.children = ([widgets.Label("Deleted"), widgets.Label("{:.1f}".format(info.x)), widgets.Label("{:.1f}".format(info.y))])
        vbox.children = vbox.children + tuple([hbox,])
        vbox
        
    elif(b.description == "Send Coordinates"):
        goal = assignment_2_2023.msg.PlanningGoal()
        goal.target_pose.pose.position.x = x_coordinate.value
        goal.target_pose.pose.position.y = y_coordinate.value
        
        hbox = widgets.HBox()

        hbox.children = ([widgets.Label("Start"), widgets.Label(str(x_coordinate.value)), widgets.Label(str(y_coordinate.value))])
        vbox.children = vbox.children + tuple([hbox,])
        vbox
        
        rospy.loginfo("Goal set to x = %f, y = %f", goal.target_pose.pose.position.x, goal.target_pose.pose.position.y)
        client.send_goal(goal)
        print("Coordinates sent with x = %.1f, y = %.1f" % (x_coordinate.value, y_coordinate.value))
        b.description = "Cancel"

In [6]:
global pub_coordinates, pub_cancel, client, info_old
info_old = Info()
info_old.x = 0.0
info_old.y = 0.0
rospy.init_node('client_node')

# create a publisher for the coordinates goal topic
pub_coordinates = rospy.Publisher('/goal/coordinates', Info, queue_size=1)

# create a publisher for the cancel goal topic
pub_cancel = rospy.Publisher('/goal/cancel', Bool, queue_size=1)

# Subscriber for feedback/status
sub_feedback = rospy.Subscriber("/reaching_goal/feedback", assignment_2_2023.msg.PlanningActionFeedback, on_feedback)
    
# Subscriber for result
sub_result = rospy.Subscriber("/reaching_goal/result", assignment_2_2023.msg.PlanningActionResult, on_result)

# sub_odom is a subscriber that receives Odometry data
sub_odom = rospy.Subscriber("/odom", Odometry, on_odom)

client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
client.wait_for_server()



[ERROR] [1712006811.332546, 4705.058000]: bad callback: <function on_odom at 0x7fa59f42c670>
Traceback (most recent call last):
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "/tmp/ipykernel_79071/106920061.py", line 18, in on_odom
    hbox_1.children = ([widgets.Label("{:.1f}".format(info.x)),])
NameError: name 'hbox_1' is not defined



True

[ERROR] [1712006811.500164, 4705.112000]: bad callback: <function on_odom at 0x7fa59f42c670>
Traceback (most recent call last):
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "/tmp/ipykernel_79071/106920061.py", line 18, in on_odom
    hbox_1.children = ([widgets.Label("{:.1f}".format(info.x)),])
NameError: name 'hbox_1' is not defined

[ERROR] [1712006811.578542, 4705.160000]: bad callback: <function on_odom at 0x7fa59f42c670>
Traceback (most recent call last):
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "/tmp/ipykernel_79071/106920061.py", line 18, in on_odom
    hbox_1.children = ([widgets.Label("{:.1f}".format(info.x)),])
NameError: name 'hbox_1' is not defined

[ERROR] [1712006811.676769, 4705.225000]: bad callback: <function on_odom at 0x7fa59f42c670>
Traceback (most recent call last):
  File "/opt/ros/noetic/lib/python3/dist-packages/rosp

In [9]:
global x_coordinate, y_coordinate, send_coordinates, vbox, hbox_1, hbox_2

x_coordinate = widgets.FloatSlider( 
value=0.0,
min=0.0, 
max=10.0, 
step=0.1,
description='X:', 
disabled=False, 
continuous_update=False, 
orientation='horizontal', 
readout=True
)
y_coordinate = widgets.FloatSlider( 
value=1.0,
min=0.0, 
max=10.0, 
step=0.1,
description='Y:', 
disabled=False, 
continuous_update=False, 
orientation='horizontal', 
readout=True
)
display(x_coordinate, y_coordinate)

send_coordinates = widgets.Button(description="Send Coordinates")
output = widgets.Output()
display(send_coordinates, output)
send_coordinates.on_click(on_coordinates_sent)

## Show interactive robot position
hbox_coordinates = widgets.HBox()
hbox_0 = widgets.HBox((widgets.Label("Coordinates:"),))
hbox_1 = widgets.HBox((widgets.Label("{:.1f}".format(0.0)),))
hbox_2 = widgets.HBox((widgets.Label("{:.1f}".format(1.0)),))
hbox_coordinates.layout = widgets.Layout(width='100%', height='100px', overflow_y='auto')
vbox_coordinates = widgets.VBox()
vbox_coordinates.children = tuple([hbox_coordinates,])
vbox_coordinates.layout = widgets.Layout(width='100%', height='100px', overflow_y='auto')
hbox_coordinates_0 = widgets.HBox([hbox_0, hbox_1, hbox_2])
hbox_coordinates.children = [hbox_coordinates_0, widgets.Label("0.0"),widgets.Label("1.0")]
display(hbox_1, hbox_2)
#textbox = TextBox('Text:', initial="Initial Text", color='gray', hovercolor='cyan')

## Show table with coordinates
# Display the tables
vbox = widgets.VBox()
hbox = widgets.HBox()

hbox.children = ([widgets.Label("Operation"), widgets.Label("X"), widgets.Label("Y")])
vbox.children = tuple([hbox,])
display(vbox)



FloatSlider(value=0.0, continuous_update=False, description='X:', max=10.0)

FloatSlider(value=1.0, continuous_update=False, description='Y:', max=10.0)

Button(description='Send Coordinates', style=ButtonStyle())

Output()

[INFO] [1712006887.375211, 4754.053000]: Goal set to x = 3.100000, y = 1.900000
Coordinates sent with x = 3.1, y = 1.9


In [ ]:
def func(x):
    return widgets.Label(x)